<a href="https://colab.research.google.com/github/harshitadd/Celestini/blob/master/Regressor_CPCB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import csv
from sklearn.linear_model import LinearRegression 
import matplotlib.pyplot as plt 
import numpy as np 
from sklearn.metrics import mean_squared_error 
import os
import math
import cv2  

In [0]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/RAW/'  #change dir to your project folder


In [24]:
#Reading JPEGs and calculating trans, entropy, contrast
temp = os.listdir('/content/gdrive/My Drive/RAW/RAW_Batch1/')
jpg_names=[]
name_jpg=[]
dummy=[]
origin_time={}           #storing original timestamps 
#loading .jpg file names (ONLY Names) from RAW_Batch1
for name in temp: 
    path = '/content/gdrive/My Drive/RAW/RAW_Batch1/' + str(name)
    if(name.endswith('.jpg')):
      try: 
        name_jpg.append(path)
        dummy=name.split('_')
        k=str(dummy[3])+str(dummy[2])+str(dummy[1])+str(dummy[4])+str(dummy[5])
        jpg_names.append(k)
        origin_time[path]=k
      except: 
        print(name)

        
#JPEG IMAGES TIMESTAMPS GENERATED AS :         
print(jpg_names)

['150620191836', '150620191831', '150620191829', '160620190530', '150620191821', '150620191811', '150620191816', '150620191803', '150620191806', '150620191756', '150620191746', '150620191736', '150620191741', '150620191733', '140620191845', '150620191709', '150620191657', '140620191842', '140620191830', '150620191613', '140620191835', '150620191554', '140620190528', '140620191856', '140620190523', '120620191903', '140620191850', '120620191905', '120620191855', '120620191806', '110620191909', '120620191722', '120620191801', '110620191900', '120620191750', '110620191828', '120620191755', '120620191742', '120620191745', '120620191735', '160620191654', '160620191648', '160620191643', '160620191638', '160620191635', '160620191628', '160620191625', '160620191619', '160620191613', '160620191608', '160620191605', '160620191600', '160620191553', '160620191548', '160620191543', '160620191539', '160620191535', '160620191530', '160620191525', '160620191518', '160620191513', '160620191510', '160620

In [25]:
#Enhancing JPEG timestamp and cpcb timestamp compatibility
compat=[]
for name in jpg_names:
  #print(name)
  hr=name[-4:-1]
  compat.append(hr)
  
  
print(len(compat))
print(compat)

712
['183', '183', '182', '053', '182', '181', '181', '180', '180', '175', '174', '173', '174', '173', '184', '170', '165', '184', '183', '161', '183', '155', '052', '185', '052', '190', '185', '190', '185', '180', '190', '172', '180', '190', '175', '182', '175', '174', '174', '173', '165', '164', '164', '163', '163', '162', '162', '161', '161', '160', '160', '160', '155', '154', '154', '153', '153', '153', '152', '151', '151', '151', '150', '150', '145', '144', '144', '143', '143', '142', '142', '142', '141', '140', '140', '140', '135', '134', '134', '134', '133', '132', '132', '131', '131', '130', '130', '130', '125', '124', '124', '123', '123', '123', '122', '121', '121', '121', '120', '120', '115', '114', '114', '113', '113', '112', '112', '111', '111', '110', '110', '105', '105', '105', '104', '103', '103', '103', '102', '102', '101', '101', '100', '100', '095', '095', '094', '094', '093', '093', '092', '091', '091', '090', '090', '085', '085', '084', '084', '083', '083', '083', '

In [0]:
def transmission(img, lt = 230, retMean = False):
  ker = np.ones((3, 3))/9.0
  imgrgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  b, g, r = cv2.split(img)
  hsv = cv2.cvtColor(imgrgb, cv2.COLOR_RGB2HSV)
  _, _, v = cv2.split(hsv)
  v_blur = cv2.filter2D(v, -1, ker)
  _,building_mask = cv2.threshold(v_blur, lt, 255, cv2.THRESH_BINARY)
  sky = cv2.bitwise_and(imgrgb, imgrgb, mask = building_mask)
  sky_mask = cv2.bitwise_not(building_mask, building_mask.copy())
  building = cv2.bitwise_and(imgrgb, imgrgb, mask = sky_mask)
  #f, ax = plt.subplots(2, 1, figsize = (15, 15))
  #ax[0].imshow(imgrgb)
  #ax[1].imshow(sky)
  #ax[2].imshow(building)
  al = max(v_blur.flatten())
  #Airlight Found. Now calculate Transmission Map
  img_norm = img/float(al)
  b, g, r = cv2.split(img_norm)
  kernel = np.ones((3,3),np.uint8)
  im_new = np.zeros((img.shape[0], img.shape[1]))
  for i in range(img.shape[0]):
    for j in range(img.shape[1]):
      im_new[i][j] = min(b[i][j], g[i][j], r[i][j])
  dcp = cv2.erode(im_new, kernel, iterations = 1)
  #ax[3].imshow(im_new, cmap = 'gray')
  #ax[4].imshow(dcp, cmap = 'gray')
  tr = np.ones((dcp.shape[0], dcp.shape[1]))
  for i in range(dcp.shape[0]):
    for j in range(dcp.shape[1]):
      tr[i][j] = 1 - dcp[i][j]
  #ax[1].imshow(tr, cmap = 'gray')
  if retMean == False:
    return tr
  elif retMean == True:
    return np.mean(tr)

In [0]:
def entropy(image):
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  hist = cv2.calcHist( [image.astype('float32')],
              [0], 
              None,
               [256], 
              [0,256] )
  h_norm = 0
  h_norm = cv2.normalize(hist, h_norm)
  ent = 0
  for p in h_norm:
    try:
      ent += p*math.log(p, 2)
    except:
      pass
    
  return (-1 * ent[0])

In [0]:
def contrast(image):
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  s = 0
  ss = 0
  mean = np.mean(image)
  x = (-1 * mean * (np.ones(image.shape)))
  ss = np.square(np.add(image, x))
  cont = np.sqrt(np.sum(ss)/float(image.size))
  return cont

In [0]:
images=[]
j=0

for name in name_jpg: 
  try:
    read = cv2.imread(name)
    read = cv2.resize(read,(32,32))
    images.append(read)
    j+=1
    print(j)
  except:
    print(name)

print(len(images))


In [0]:
trans=[]
cont=[]
entropy_jpg=[]
for img in images:
  try:
    trans.append(transmission(img, retMean  = True))
    cont.append(contrast(img))
    entropy_jpg.append(entropy(img))
  except: 
    pass
    
print(trans)
print(len(trans))
print(cont)
print(len(cont))
print(entropy_jpg)
print(len(entropy_jpg))

In [32]:
#loading the cpcb csv
cpcb_time={}
k=0
with open('/content/cpcb_historic(11_16).csv') as cpcb: 
  reader=csv.reader(cpcb, delimiter = ',')
  for row in reader:
    k+=1
    if(k>=18):
      dummy=row[0]
      dummy=dummy.split('-')
      temp = dummy[2].split(' ')
      n = temp[1].split(':')
      dummy = str(dummy[0])+str(dummy[1])+str(temp[0]) + str(n[0]) + str(n[1])
      cpcb_time[dummy]=row[2]
      
      
print(cpcb_time)
      

{'110620190000': '122', '110620190015': '120', '110620190030': '118', '110620190045': '113', '110620190100': '110', '110620190115': '107', '110620190130': '102', '110620190145': '95', '110620190200': '89', '110620190215': '86', '110620190230': '87', '110620190245': '93', '110620190300': '99', '110620190315': '106', '110620190330': '113', '110620190345': '117', '110620190400': '122', '110620190415': '125', '110620190430': '128', '110620190445': '134', '110620190500': '140', '110620190515': '149', '110620190530': '159', '110620190545': '167', '110620190600': '173', '110620190615': '176', '110620190630': '178', '110620190645': '180', '110620190700': '181', '110620190715': '182', '110620190730': '186', '110620190745': '192', '110620190800': '201', '110620190815': '211', '110620190830': '219', '110620190845': '223', '110620190900': '230', '110620190915': '237', '110620190930': '245', '110620190945': '254', '110620191000': '256', '110620191015': '250', '110620191030': '235', '110620191045': 

In [33]:
final={}
for key in jpg_names: 
  for t in cpcb_time: 
    if(t==key):
      final[t]=cpcb_time[t]
      
print(len(jpg_names))
print(len(cpcb_time))
print(len(final))
print(final)


712
577
38
{'160620190530': '25', '140620191845': '42', '140620191830': '42', '110620191900': '69', '120620191745': '61', '160620191600': '28', '160620191530': '31', '160620191500': '30', '160620191400': '29', '160620191300': '26', '160620191230': '28', '160620191200': '27', '160620191145': '24', '160620191030': '24', '160620191000': '32', '160620190830': '33', '160620190815': '33', '160620190730': '30', '160620190630': '21', '150620191400': '54', '150620190800': '106', '150620190615': '63', '150620190515': '69', '140620191815': '44', '140620191730': '45', '140620191200': '46', '140620191015': '50', '140620190945': '53', '140620190815': '50', '140620190645': '48', '130620190830': '67', '130620190800': '68', '120620191700': '60', '120620191415': '58', '120620191330': '59', '120620191200': '47', '120620191015': '44', '110620191845': '70'}
